# use estimator
**estimator을 기본으로 하는 간단한 logistic regression을 만든다.**  
**source 설명 이후 실습을 진행한다.**

In [1]:
samples = ['너 오늘 이뻐 보인다',
          '나는 오늘 기분이 더러워',
          '끝내주는데, 좋은 일이 있나봐',
          '나 좋은 일이 생겼어',
          '아 오늘 진짜 짜증나',
          '환상적인데, 정말 좋은거 같아']

targets =[[1], [0], [1], [1], [0], [1]]

In [2]:
import os
import tensorflow as tf
import numpy as np

from tensorflow.keras import preprocessing

In [3]:
tokenizer = preprocessing.text.Tokenizer() 
tokenizer.fit_on_texts(samples) 
sequences = tokenizer.texts_to_sequences(samples)
sequences = preprocessing.sequence.pad_sequences(sequences, maxlen = 6, padding='post')

targets = np.array(targets)

word_index = tokenizer.word_index

print("index text data : \n", sequences)
print("shape of sequences:", sequences.shape)

index text data : 
 [[ 4  1  5  6  0  0]
 [ 7  1  8  9  0  0]
 [10  2  3 11  0  0]
 [12  2  3 13  0  0]
 [14  1 15 16  0  0]
 [17 18 19 20  0  0]]
shape of sequences: (6, 6)


In [4]:
epoch_size = 100
batch_size = 2

def train_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((sequences, targets))
    dataset = dataset.repeat(epoch_size)
    dataset = dataset.batch(batch_size)
    dataset = dataset.shuffle(len(sequences))

    return dataset

In [5]:
vocab_size = len(word_index) + 1
emb_size = 128

def model_fn(features, labels, mode):
    
    print(features)
    embed_input = tf.keras.layers.Embedding(vocab_size, emb_size)(features)
    embed_input = tf.reduce_mean(embed_input, axis=-1)
    
    hidden_layer = tf.keras.layers.Dense(128, activation=tf.nn.relu)(embed_input)
    output_layer = tf.keras.layers.Dense(1)(hidden_layer)
    output = tf.nn.sigmoid(output_layer)
    
    loss = tf.losses.mean_squared_error(output, labels)

    global_step = tf.train.get_global_step()
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss, global_step)

    return tf.estimator.EstimatorSpec(
              mode=mode,
              train_op=train_op,
              loss=loss)

In [6]:
DATA_OUT_PATH = './data_out/'

if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)

estimator = tf.estimator.Estimator(model_fn = model_fn, model_dir = DATA_OUT_PATH + 'checkpoint/dnn')

In [7]:
tf.logging.set_verbosity(tf.logging.INFO)

AttributeError: module 'tensorflow' has no attribute 'logging'

In [ ]:
estimator.train(train_input_fn)